In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from scipy.stats.stats import ttest_ind
import warnings

warnings.filterwarnings('ignore')

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'studentsperformance'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

education_df = pd.read_sql_query('select * from studentsperformance',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

1. Are there any differences between the genders, ethnicities, and parental level of education with respect to their performances in exams?


2. Are there any differences between the lunch types with respect to their performances in exams? If there are, how do you explain this?


3. Does the test preparation course seem to have an effect on the exam performances?


4. Which 2 exam scores are most correlated with each other?

### 1. Are there any differences between the genders, ethnicities and parental level of education with respect to their performances in exams?

In [4]:
education_df.groupby("gender").mean()[["math score", "reading score", "writing score"]]

,math score,reading score,writing score
gender,,,
female,63.633205,72.608108,72.467181
male,68.728216,65.473029,63.311203


In [6]:
ttest_ind(education_df[education_df.gender == "female"][["math score", "reading score", "writing score"]], education_df[education_df.gender == "male"][["math score", "reading score","writing score"]])

Ttest_indResult(statistic=array([-5.38324587,  7.95930801,  9.97955791]), pvalue=array([9.12018555e-08, 4.68053874e-15, 2.01987771e-22]))

P-values are more than .05, so they are significant difference.

In [8]:
education_df.groupby("race/ethnicity").mean()[["math score", "reading score", "writing score"]]

,math score,reading score,writing score
race/ethnicity,,,
group A,61.629213,64.674157,62.674157
group B,63.452632,67.352632,65.600000
group C,64.463950,69.103448,67.827586
group D,67.362595,70.030534,70.145038
group E,73.821429,73.028571,71.407143


In [21]:
ethnicities = education_df["race/ethnicity"].unique()
grouped_df = education_df.groupby("race/ethnicity")

for var in ["math score", "reading score", "writing score"]:
    print("============================")
    print("Comparisons for variable: {}".format(var))
    print("============================")
    for i in range(0, len(ethnicities)):
        for j in range(i+1, len(ethnicities)):
            print("t-test between groups {0} and {1}:".format(ethnicities[i],ethnicities[j]))
            
            print(ttest_ind(education_df[education_df["race/ethnicity"]== ethnicities[i]][var],education_df[education_df["race/ethnicity"] == ethnicities[j]][var]))

Comparisons for variable: math score
t-test between groups group B and group C:
Ttest_indResult(statistic=-0.7315669893534263, pvalue=0.4647708939167453)
t-test between groups group B and group A:
Ttest_indResult(statistic=0.9355076279747488, pvalue=0.3503415961659957)
t-test between groups group B and group D:
Ttest_indResult(statistic=-2.82845539712675, pvalue=0.004886197137104194)
t-test between groups group B and group E:
Ttest_indResult(statistic=-6.007397050552227, pvalue=5.007946047497971e-09)
t-test between groups group C and group A:
Ttest_indResult(statistic=1.5997220303217299, pvalue=0.11043810745588042)
t-test between groups group C and group D:
Ttest_indResult(statistic=-2.41858624746011, pvalue=0.015888349556016285)
t-test between groups group C and group E:
Ttest_indResult(statistic=-6.127642520822135, pvalue=1.927238849552764e-09)
t-test between groups group A and group D:
Ttest_indResult(statistic=-3.3466982429712298, pvalue=0.0009068636053543751)
t-test between groups

### 2. Are there any differences between the lunch types with respect to their performances in exams? If there are, how do you explain this?

In [22]:
education_df.groupby("lunch").mean()[["math score", "reading score", "writing score"]]

,math score,reading score,writing score
lunch,,,
free/reduced,58.921127,64.653521,63.022535
standard,70.034109,71.654264,70.823256


In [23]:
ttest_ind(education_df[education_df.lunch == "free/reduced"][["math score", "reading score", "writing score"]],
         education_df[education_df.lunch == "standard"][["math score", "reading score", "writing score"]])

Ttest_indResult(statistic=array([-11.83718047,  -7.45105647,  -8.0097842 ]), pvalue=array([2.41319560e-30, 2.00279665e-13, 3.18618958e-15]))

### 3. Does the test preparation course seem to have an effect on the exam performances?

In [24]:
education_df.groupby("test preparation course").mean()[["math score", "reading score", "writing score"]]

,math score,reading score,writing score
test preparation course,,,
completed,69.695531,73.893855,74.418994
none,64.077882,66.534268,64.504673


In [29]:
ttest_ind(education_df[education_df["test preparation course"] =="completed"][["math score", "reading score", "writing score"]], education_df[education_df["test preparation course"] == "none"][["math score", "reading score", "writing score"]])

Ttest_indResult(statistic=array([ 5.70461642,  7.87166354, 10.40917344]), pvalue=array([1.53591346e-08, 9.08178334e-15, 3.68529174e-24]))

### 4. Which two exam scores are correlated the most with each other?

In [31]:
education_df[["math score", "reading score", "writing score"]].corr()

,math score,reading score,writing score
math score,1.000000,0.817580,0.802642
reading score,0.817580,1.000000,0.954598
writing score,0.802642,0.954598,1.000000


In [ ]:
Reading and writing are more correlated than any other combination